In [1]:
import collector

In [2]:
main = collector.Collector('signet')

In [3]:
main._setDart('0e6e1eeb01f20f0ec7e5134b030e704764dd930e')
main._setMaria()

In [4]:
def checkFS(code, df):
    res = df[df['종목코드'] == code]['FS'].iloc[0]
    return res

def saveData(tablename, data):
    if data is None:
        return
    main.maria.insertData(tablename, data)
    main.maria.commitDB()
    
def getFloatData(data):
    return [float(d.replace(',',''))/100000000 for d in data]

def getDGData(code, df):
    fs = main.getDartFs(code, True, 2019, '11014')
    if fs is None:
        print('{}는 api에 데이터가 없습니다'.format(code))
        main.writeError('error_dart', code, 'api에 데이터가 없음')
        return fs
    check_fs = checkFS(code, df)
    if check_fs == '연결':
        df = fs[fs['fs_div'] == 'CFS']
    else:
        df = fs[fs['fs_div'] == 'OFS']
    try:
        dg_data = getFloatData(list(df[df['account_nm'] == '당기순이익'][['frmtrm_amount', 'thstrm_amount']].iloc[0]))
    except:
        try:
            if check_fs == '연결':
                df = fs[fs['fs_div'] == 'OFS']
                print('{}를 개별에서 가져왔습니다'.format(code))
                main.writeError('error_dart', code, '연결 당기 데이터에 없어서 개별 데이터에서 가져옴')
                dg_data = getFloatData(list(df[df['account_nm'] == '당기순이익'][['frmtrm_amount', 'thstrm_amount', 'thstrm_amount']].iloc[0]))
            else:
                df = fs[fs['fs_div'] == 'CFS']
                print('{}를 연결에서 가져왔습니다'.format(code))
                main.writeError('error_dart', code, '개별 당기 데이터에 없어서 연결 데이터에서 가져옴')
                dg_data = getFloatData(list(df[df['account_nm'] == '당기순이익'][['frmtrm_amount', 'thstrm_amount', 'thstrm_amount']].iloc[0]))
        except:
            print('{}의 finstate에는 [당기순이익]이 없습니다'.format(code))
            main.writeError('error_dart', code, '[당기순이익]데이터가 없음')
            return None
    return dg_data

In [5]:
data = {}
years = ['2017', '2018', '2019', '2020']
for y in years:
    data['fnguide{}'.format(y)] = main.maria.showData('select * from fnguide{}'.format(y))
code_list = list(data['fnguide2017']['종목코드'])

# 당기순이익 3Q

In [6]:
done_list = main.getDoneList('new_dart_dg')
std_df = data['fnguide2017']
for code in code_list[10:]:
    if code in done_list:
        continue
    print(code)
    dg_data = getDGData(code, std_df)
    if dg_data is None:
        continue
    saveData(tablename='new_dart_dg', data=tuple([code]+dg_data))

# Signet

In [7]:
# import time
# from bs4 import BeautifulSoup
# import FinanceDataReader as fdr
# import pandas as pd

# code = '000660'
# std_df = data['fnguide2017']

# main._setChrome()
# main.signetCollect()